## Lab 2, Continuation - Code & Comments
Elysa Strunin  
October 2018  

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from pandas.io.json import json_normalize

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from sklearn.pipeline import make_pipeline

from scipy import stats

import matplotlib
import matplotlib.pyplot as plt

### Data Preparation

In [9]:
dose_response = pd.read_csv('v17_fitted_dose_response.csv')
dose_response.head()

,Dataset_version,IC50_Results_ID,COSMIC_ID,DRUG_ID,MAX_CONC_MICROMOLAR,LN_IC50,AUC,RMSE
0,17,335,924100,1026,1.0,0.72,0.90,0.11
1,17,336,924100,1028,2.0,2.66,0.96,0.18
2,17,337,924100,1029,2.0,3.34,0.97,0.08
3,17,338,924100,1030,10.0,5.16,0.98,0.09
4,17,339,924100,1031,0.2,-4.33,0.51,0.09


In [10]:
uniques = pd.Series(dose_response['DRUG_ID'].unique())

expr = pd.read_table('Cell_line_COSMIC_ID_gene_expression_transposed_clean.tsv', header=None)

In [11]:
header = pd.read_table('Cell_line_RMA_proc_basalExp_transposed.tsv', nrows = 2)

more_info = header

In [12]:
expr.iloc[:,0] = expr.iloc[:,0].astype(int)
expr.columns = header.columns
expr = expr.rename(index=str, columns={"GENE_SYMBOLS": "COSMIC_ID"})

### Linear Regression

In [14]:
rmse_all = []
model_coef_all = []
model_int_all = []
pca_all = []
means_all = []
stdevs_all = []

for drug in dose_response['DRUG_ID'].unique():
    subset = dose_response[ dose_response['DRUG_ID']== drug ].loc[:,['COSMIC_ID','LN_IC50']] # 'AUC', 'RMSE' - remove auc, rmse if not filtering
    expre = expr.merge(subset, on='COSMIC_ID', how='left', sort=False)

    #remove the id's 
    expre = expre.iloc[:,1:]
    expr_data = expre.copy() 
    expr_target = expre.loc[:,'LN_IC50'] #just testing one target here
    del expr_data['LN_IC50']

    # there will be a lot of na's in ic50. remove them.
    mask = np.array(~expr_target.isnull())
    expr_target = expr_target[mask]
    expr_data = pd.DataFrame(expr_data)[mask]
      
    # CV k= 5
    rmse_drug = []
    model_coef_drug = []
    model_int_drug = []
    pca_drug = []
    means_drug = []
    stdevs_drug = []
    
    for i in np.arange(0,5) :
        
        train_data, test_data, train_target, test_target = train_test_split( expr_data, expr_target, test_size=0.2)
        # rownames are cosmic ids
        # colnames are gene expression

        if train_data.shape[1] ==0 or test_data.shape[1] ==0 :
            rmse_drug.append(np.nan)
            
        else:

            #fit on training data only
            scaled_train = preprocessing.scale(train_data)
            scaled_test = preprocessing.scale(test_data)
            
            means_drug.append( train_data.apply(np.mean) )
            stdevs_drug.append( train_data.apply(np.std) )
            
            if i==0:
                pca = PCA(.90)
                train_img = pca.fit_transform(scaled_train)
                n_comp = pca.n_components_
            
            #this sets as constant the dimensions throughout a cv
            else:
                pca = PCA(n_comp)
                train_img = pca.fit_transform(scaled_train)
            
            test_img = pca.transform(scaled_test)

            train_target = np.ravel(train_target)

            linRegr = LinearRegression()
            linRegr.fit(train_img, train_target)

            pred = linRegr.predict(test_img)

            mask = np.array(~test_target.isnull())
            test_targ = test_target[mask]
            pred = pd.DataFrame(pred)[mask]

            if test_targ.shape[0] ==0 or pred.shape[0] ==0 : 
                rmse_drug.append(np.nan)
                model_coef_drug.append(np.nan)
                model_int_drug.append(np.nan)
                pca_drug.append(np.nan)
  
            else:        
                rmse_drug.append( np.sqrt(mean_squared_error(test_targ, pred) ))
                model_coef_drug.append(linRegr.coef_)
                model_int_drug.append(linRegr.intercept_)
                pca_drug.append(pca.components_)

    rmse_all.append(np.mean(rmse_drug)) 
    model_coef_all.append(np.mean(model_coef_drug, axis=0)) 
    model_int_all.append(np.mean(model_int_drug, axis=0)) 
    pca_all.append(np.mean(pca_drug, axis=0)) 
    means_all.append(np.mean(means_drug, axis=0))
    stdevs_all.append(np.mean(stdevs_drug, axis=0))


### Model Comparisons (RMSE)

In [170]:
def drugs_per_model(cosmic_id):
    
    ln_ic50 = []
    
    #subset the data
    subset = dose_response[ dose_response['COSMIC_ID']== cosmic_id ].loc[:,['DRUG_ID','LN_IC50']] 
    expre =  expr[ expr['COSMIC_ID']== cosmic_id ]    

    data = expre.iloc[0:1,1:]


    for drug_id in uniques:

        #get its index
        i = uniques[uniques == drug_id].index[0]

        #scaling
        data0 = (data - means_all[i]) / stdevs_all[i] #confirm that vec div by vec is vec

        #pca transformation
        data0 = data0 @ pca_all[i].T

        #prediction
        val = np.sum(model_coef_all[i] * data0, axis=1) + model_int_all[i]
        ln_ic50.append( val[0] )

    ln_ic50 = pd.Series(ln_ic50)
    
    preds = pd.DataFrame({'model_preds': ln_ic50, 'DRUG_ID': pd.Series(uniques) } ) 

    comparison = preds.merge(subset, on='DRUG_ID', how='left', sort=False)
    
    comparison.index = comparison.loc[:, 'DRUG_ID']
    
    del comparison['DRUG_ID']
    
    return comparison

In [171]:
#Compare this list for each cell line to the experimental data in the GDSC experimental results. 

#Come up with a score to compare your rank ordered list to the rank ordering of IC50 from the Dose

In [172]:
df = drugs_per_model(924100)

In [173]:
#Report the top ten drugs predicted to be sensitive for each cell line in order of best to worst. 
#lower ic50 means more greater sensitivity
df.sort_values('model_preds').head(10)

,model_preds,LN_IC50
DRUG_ID,,
104,-4.981976,NaN
201,-4.695227,-4.94
1007,-4.570078,-5.70
140,-4.137135,-4.59
268,-3.707443,-5.18
1003,-3.572944,-4.73
1004,-3.507021,-4.37
1494,-3.487913,-4.75
1031,-3.116130,-4.33


In [167]:
df = df.dropna()

In [175]:
#test whether the difference in model prediction and observation, per drug, has mean zero
#The Wilcoxon signed-rank test tests the null hypothesis that two related paired samples come 
#from the same distribution. In particular, it tests whether the distribution of the differences
#x - y is symmetric about zero. 
#not sure if these qualify as dependent pairs...

#p-value is sufficiently small to reject the null hypothesis
#that the pairs come from the same distribution.

stats.wilcoxon(x=np.ravel(df[['model_preds']]), y=np.ravel(df[['LN_IC50']]), zero_method='wilcox', correction=False)

WilcoxonResult(statistic=9569.0, pvalue=0.0024758060061318464)

In [207]:
#But, we can imagine a scenario in which we reject the null for the above test
#even when the drug lists are in exactly the same order (just very different IC50 values)

#top10 in top10 ? get a percent
#top50 in top50 ? get a percent
#top100 in top100 ? get a percent
#top200 in top200 ? get a percent
#average the percents

#.... obviuously, these cutoffs are arbitrary... 
#a more granular test would still, in this direction, prioritize the ordering of the top of the list 
#over the bottom of the list


In [190]:
def general_similarity(top_number):
    x = df.loc[:, 'model_preds'].sort_values()
    model_preds_top = x.index[:top_number].values

    y = df.loc[:, 'LN_IC50'].sort_values()
    LN_IC50_top = y.index[:top_number].values

    place = []
    
    for element in model_preds_top:
        place.append(element in LN_IC50_top)
        
    return (np.sum(place) / len(place) )


In [206]:
general_similarity(25)

0.88

In [203]:
df.loc[:, 'model_preds'].sort_values().head(10)

DRUG_ID
201    -4.695227
1007   -4.570078
140    -4.137135
268    -3.707443
1003   -3.572944
1004   -3.507021
1494   -3.487913
1031   -3.116130
283    -2.870295
180    -2.793550
Name: model_preds, dtype: float64

In [204]:
df.loc[:, 'LN_IC50'].sort_values().head(10)

DRUG_ID
1007   -5.70
180    -5.42
268    -5.18
201    -4.94
1494   -4.75
1003   -4.73
140    -4.59
1004   -4.37
1031   -4.33
283    -4.13
Name: LN_IC50, dtype: float64

In [114]:
#do the models check out wrt rmse?

rmse_all = pd.Series(rmse_all)
rmse_all.index = dose_response['DRUG_ID'].unique()[:]
rmse_all.sort_values().head(10)

1262    0.453157
150     0.481165
266     0.505088
1264    0.534266
341     0.571337
205     0.600437
1502    0.604841
312     0.606107
91      0.620644
193     0.670032
dtype: float64